In [9]:
import numpy as np
import cupy as cp
import cupyx.scipy.sparse as sp
from numpy.random import default_rng
import pandas as pd
import pickle
import time
import sys
import os
import math
np.set_printoptions(threshold=sys.maxsize)

global maxInterference
maxInterference = 0.0

global trackTime
trackTime = 0


n = 20000
p = 512 / 99999
d = (int) (p * (n-1))
k = (int) (d * 32/512)
r_approx = (int) ((n / 100000) * (pow(2, 16) * k / d))
M = cp.empty(shape=(0, n), dtype='bool')
rng = default_rng(seed=42)

print(n)
print(d)
print(k)
print(r_approx)

def create_sparse_gnp_graph() -> sp.csr_matrix:
        
    num_edges = rng.binomial(n * (n - 1) // 2, p)
    sources = cp.array(rng.integers(0, n, (int)(num_edges*2)))
    targets = cp.array(rng.integers(0, n, (int)(num_edges*2)))
    mask = sources != targets
    data = cp.ones_like(sources[mask])
    adj_matrix = sp.coo_matrix((data, (targets[mask], sources[mask])), (n, n), dtype='bool')
    adj_matrix = adj_matrix.tocsr()

    del sources
    del targets
    del data

    return adj_matrix

g = create_sparse_gnp_graph().toarray()

20000
102
6
771


In [10]:
def random_memory():
    temp = np.zeros(n, dtype='bool')
    temp[:r_approx] = True
    rng.shuffle(temp)
    return cp.array(temp)

def SJOIN_interference_check(A, i):
    global maxInterference
    if i <= 0:
        return 0
    this_max = cp.max(cp.sum(cp.logical_and(M[:i], A), axis=1)) / cp.sum(A)
    if this_max > maxInterference:
        maxInterference = this_max
    return this_max

def SJOIN_one_step(A):
    fired_neighbors = cp.sum(g[A], axis=0)
    counts = cp.zeros((int)(cp.max(fired_neighbors))+1, dtype=int)
    for i in range(n):
        for j in range((int)(fired_neighbors[i])+1):
            counts[j] = counts[j]+1

    idx = len(counts)-1
    for i in range(len(counts)):
        if counts[i] < r_approx:
            if i == 0:
                idx = 0
            else:
                if counts[i-1] - r_approx < r_approx - counts[i]:
                    idx = i-1
                else:
                    idx = i
            break
    
    B = cp.zeros(n, dtype='bool')
    for i in range(n):
        if fired_neighbors[i] >= idx:
            B[i] = True

    print("{} : {}".format(idx, cp.sum(B)))

    return B

def SJOIN_long_sequence(num):
    global maxInterference
    global M

    df = None

    if os.path.isfile('output.xlsx'):
        print("File found")
        df = pd.read_excel('output.xlsx')
        maxInterference = df.loc[len(df.index)-1][0]
    else:
        print("Create new file")
        columns = ["Max Interference", "Interference", "Memory"]
        df = pd.DataFrame(columns=columns)
        maxInterference = 0

    if len(M) == 0:
        M = cp.append(M, [random_memory()], axis=0)
        new_row = [0, 0, r_approx]
        df.loc[len(df.index)] = new_row


    memory_count = 1
    while True:
        print("Memory {}".format(memory_count))

        A = M[-1]
        B = SJOIN_one_step(A)
        
        this_max = SJOIN_interference_check(B, len(M)-1)

        M = cp.append(M, [B], axis=0)

        new_row = [(float)(maxInterference), (float)(this_max), (int)(cp.sum(B))]
        df.loc[len(df.index)] = new_row

        if this_max >= 0.5 or memory_count >= num:
            df.to_excel('output.xlsx', index=False)
            return
        
        memory_count = memory_count + 1

In [11]:
with open('M.pkl', 'wb') as f:
    pickle.dump(M, f)
np.save("rng.npy", rng.bit_generator.state)

In [13]:
with open('M.pkl', 'rb') as f:
    M = pickle.load(f)
rng.bit_generator.__setstate__(np.load("rng.npy" ,allow_pickle='TRUE').item())


SJOIN_long_sequence(num=5)


with open('M.pkl', 'wb') as f:
    pickle.dump(M, f)
np.save("rng.npy", rng.bit_generator.state)

File found
Memory 1


/tmp/ipykernel_159236/2603431598.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  maxInterference = df.loc[len(df.index)-1][0]


9 : 568
Memory 2
7 : 565
Memory 3
7 : 541
Memory 4
7 : 458
Memory 5
6 : 629
